In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import pandas as pd

! pip install plotly
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [2]:
PATH = 'data_preprocess.csv'

## Preprocessing

In [2]:
sp500 = pd.read_csv(PATH)

NameError: name 'PATH' is not defined

In [4]:
# Checking null
sp500.isnull().sum()

Unnamed: 0      0
time            0
open            0
high            0
low             0
close           0
Volume          0
Volume MA       0
MA              0
Histogram       0
MACD            0
Signal          0
RSI             0
RSI-based MA    0
dtype: int64

In [5]:
#  Checking nan
sp500.isna().sum()

Unnamed: 0      0
time            0
open            0
high            0
low             0
close           0
Volume          0
Volume MA       0
MA              0
Histogram       0
MACD            0
Signal          0
RSI             0
RSI-based MA    0
dtype: int64

In [6]:
#  Dropnan
sp500.dropna(inplace = True)
sp500.isna().any()

Unnamed: 0      False
time            False
open            False
high            False
low             False
close           False
Volume          False
Volume MA       False
MA              False
Histogram       False
MACD            False
Signal          False
RSI             False
RSI-based MA    False
dtype: bool

In [7]:
sp500['time'] = pd.to_datetime(sp500.time)

In [8]:
sp500.shape

(20743, 14)

### Data Exploration

In [ ]:
#group 7 days into a group to predict the next day
print("Starting date: ",sp500.iloc[0][0])
print("Ending date: ", sp500.iloc[-1][0])
print("Duration: ", sp500.iloc[-1][0]-sp500.iloc[0][0])

Starting date:  88
Ending date:  20830
Duration:  20742


In [10]:
# sp500.to_csv('data_preprocess.csv')

## Machine Learning

In [10]:
from utils.utils import create_data_train, read_csv
from configs.config import *
from utils.preprocess import Preprocessing
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score 

In [11]:
path = 'data_preprocess.csv'
ts_config = TSConfig()
frame = read_csv(path, ts_config.train_features, ts_config.pred_features, return_np = True)
preprocess = Preprocessing()
ML = True

data_preprocess.csv


In [12]:
feature = frame['feature']
target = frame['target']

In [13]:
target.shape

(20743, 5)

In [14]:
#   Scaling data
feature, (feature_max_scale, feature_min_scale) = preprocess.apply_minmax_scale(feature)

In [15]:
target, (target_max_scale, target_min_scale) = preprocess.apply_minmax_scale(target)

In [16]:
train_test_ratio = 0.8
seq_steps        = 7

In [17]:
X, Y = create_data_train(feature, target, seq_steps)
if ML:
    X = X.detach().cpu().numpy()
    Y = Y.detach().cpu().numpy()

In [18]:
split_idx = int(X.shape[0] * train_test_ratio)


In [19]:
#   Predict high
x_train, y_train = X[:split_idx, :, :], Y[:split_idx, :]
x_test, y_test   = X[split_idx:, :, :], Y[split_idx:, :]

print(y_train.shape, y_test.shape)

(16587, 5) (4147, 5)


In [20]:
print(f'Number of traning samples = {x_train.shape[0]}\nNumber of testing samples = {y_test.shape[0]}')
print(x_train.shape)

Number of traning samples = 16587
Number of testing samples = 4147
(16587, 7, 12)


In [21]:
def max_min_inverse(inp, max_scale, min_scale):
    print(max_scale)
    return (inp * (max_scale - min_scale)) + min_scale

In [22]:
#   Predict categories
"""
'high': 0
'low': 1
'MA:   2
'MACD': 3
'open': 4
'close': 5
'Volume': 6
'Volume MA': 7
'MA': 8
'Histogram': 9
'MACD': 10
'Signal': 11
"""
category = 8
x_train_ml = x_train[:, :, category]
x_test_ml = x_test[:, :, category]
y_train_ml = y_train[:, category]
y_test_ml  = y_test[:,category]

print(f'Feature selection = {ts_config.pred_features[category]}')
print(f'Train shape: {x_train_ml.shape}, {y_train_ml.shape}\nTest shape: {x_test_ml.shape}, {y_test_ml.shape}')

IndexError: index 8 is out of bounds for axis 1 with size 5

### Support Vector Regressor

In [67]:
from sklearn.svm import SVR
from configs.config import BASEDIR
import pickle

In [55]:
def evaluate_SVR(x_train, y_train, x_test, y_test, C, gamma, category):
    svr_rbf = SVR(kernel = 'rbf', C= 1e2, gamma= 0.1)
    svr_rbf.fit(x_train, y_train)

    train_predict=svr_rbf.predict(x_train)
    test_predict=svr_rbf.predict(x_test)

    y_test_rescale = max_min_inverse(y_test, target_max_scale[:, category], target_min_scale[:, category])
    pred_rescale   = max_min_inverse(test_predict, target_max_scale[:, category], target_min_scale[:, category])

    train_predict = train_predict.reshape(-1,1)
    test_predict = test_predict.reshape(-1,1)
    
    mean_sqr_err = mean_squared_error(y_test_rescale, pred_rescale)
    mean_abs_err = mean_absolute_error(y_test_rescale, pred_rescale)
    r_square     = r2_score(y_test_rescale, pred_rescale)
    print("Train data prediction:", train_predict.shape)
    print("Test data prediction:", test_predict.shape)
    print(f'Prediction of {ts_config.pred_features[category]}')
    print(f'MAE = {mean_abs_err}\nMSE = {mean_sqr_err}\nR square = {r_square}')

    results = {
        'prediction_' + ts_config.pred_features[category]: pred_rescale,
        'ground_truth': y_test_rescale,
        'MSE': mean_sqr_err,
        'MAE': mean_abs_err,
        'R2': r_square,
        'RMSE': np.sqrt(mean_sqr_err)
    }

    print(results)
    save_pkl_path = BASEDIR / f'save/SVR_{ts_config.pred_features[category]}.pkl'
    with open(save_pkl_path, mode=  'wb') as writer:
        pickle.dump(results, writer)
    writer.close()

In [ ]:
evaluate_SVR(x_train_ml, y_train_ml, x_test_ml, y_test_ml, 1e2, 1e-1, category)

### Random Forest

In [23]:
from sklearn.ensemble import RandomForestRegressor
import pickle

In [24]:
def evaluate_randomfr(x_train, y_train, x_test, y_test, C, gamma, category):


    regressor = RandomForestRegressor(n_estimators = 100, random_state = 0)
    regressor.fit(x_train, y_train)

    train_predict=regressor.predict(x_train)
    test_predict=regressor.predict(x_test)

    y_test_rescale = max_min_inverse(y_test, target_max_scale[:, category], target_min_scale[:, category])
    pred_rescale   = max_min_inverse(test_predict, target_max_scale[:, category], target_min_scale[:, category])

    train_predict = train_predict.reshape(-1,1)
    test_predict = test_predict.reshape(-1,1)
    
    mean_sqr_err = mean_squared_error(y_test_rescale, pred_rescale)
    mean_abs_err = mean_absolute_error(y_test_rescale, pred_rescale)
    r_square     = r2_score(y_test_rescale, pred_rescale)
    print("Train data prediction:", train_predict.shape)
    print("Test data prediction:", test_predict.shape)
    print(f'Prediction of {ts_config.pred_features[category]}')
    print(f'MAE = {mean_abs_err}\nMSE = {mean_sqr_err}\nR square = {r_square}')

    results = {
        'prediction_' + ts_config.pred_features[category]: pred_rescale,
        'ground_truth': y_test_rescale,
        'MSE': mean_sqr_err,
        'MAE': mean_abs_err,
        'R2': r_square,
        'RMSE': np.sqrt(mean_sqr_err)
    }

    print(results)
    save_pkl_path = BASEDIR / f'save/RandomF/rfr_{ts_config.pred_features[category]}.pkl'
    with open(save_pkl_path, mode=  'wb') as writer:
        pickle.dump(results, writer)
    writer.close()

In [25]:
evaluate_randomfr(x_train_ml, y_train_ml, x_test_ml, y_test_ml, 1e2, 1e-1, category)

[4749.065393]
[4749.065393]
Train data prediction: (16587, 1)
Test data prediction: (4147, 1)
Prediction of MA
MAE = 578.3874233819076
MSE = 424932.8017719182
R square = -3.700542999875011
{'prediction_MA': array([3560.30940787, 3560.30940787, 3560.30940787, ..., 3560.30940787,
       3560.30940787, 3560.30940787]), 'ground_truth': array([3562.29623436, 3563.2345051 , 3564.23918094, ..., 4001.32070697,
       4002.93473328, 4004.43386442]), 'MSE': 424932.8017719182, 'MAE': 578.3874233819076, 'R2': -3.700542999875011, 'RMSE': 651.8686997946122}


### ARIMA

In [3]:
from statsmodels.tsa.arima_model import ARIMA

## Stacked LSTM

In [55]:
import tensorflow as tf 
from tensorflow.keras.layers import LSTM, Dense, GRU
from tensorflow.keras import Sequential

In [40]:
tf.keras.backend.clear_session()
model=Sequential()
model.add(LSTM(16,return_sequences=True,input_shape=(ts_config.n_steps, ts_config.n_features)))
model.add(LSTM(16))
model.add(Dense(ts_config.n_classes))
model.compile(loss='mean_squared_error',optimizer='adam')

In [41]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 7, 16)             1856      
                                                                 
 lstm_1 (LSTM)               (None, 16)                2112      
                                                                 
 dense (Dense)               (None, 5)                 85        
                                                                 
Total params: 4,053
Trainable params: 4,053
Non-trainable params: 0
_________________________________________________________________


In [42]:
x_train_dl = X[:split_idx, :, :]
y_train_dl = Y[:split_idx, :]
x_test_dl  = X[split_idx: , :, :]
y_test_dl  = Y[split_idx:, :]

print(f'Train samples: {x_train_dl.shape} -- Test samples: {x_test_dl.shape}')
print(f'Train targets: {y_train_dl.shape} -- Test targets: {y_test_dl.shape}')

Train samples: (16587, 7, 12) -- Test samples: (4147, 7, 12)
Train targets: (16587, 5) -- Test targets: (4147, 5)


In [43]:
model.fit(x_train_dl,y_train_dl,validation_data=(x_test_dl,y_test_dl),epochs=100,batch_size=512,verbose=1)
model.save(f'save/StackedLSTM/ckpt/lstm.h5')

Epoch 1/100
33/33 [==============================] - 4s 31ms/step - loss: 0.0646 - val_loss: 0.1348
Epoch 2/100
33/33 [==============================] - 0s 15ms/step - loss: 0.0099 - val_loss: 0.0616
Epoch 3/100
33/33 [==============================] - 0s 15ms/step - loss: 0.0018 - val_loss: 0.0185
Epoch 4/100
33/33 [==============================] - 0s 14ms/step - loss: 4.1589e-04 - val_loss: 0.0157
Epoch 5/100
33/33 [==============================] - 0s 15ms/step - loss: 3.3926e-04 - val_loss: 0.0149
Epoch 6/100
33/33 [==============================] - 0s 15ms/step - loss: 2.9463e-04 - val_loss: 0.0143
Epoch 7/100
33/33 [==============================] - 0s 15ms/step - loss: 2.5846e-04 - val_loss: 0.0134
Epoch 8/100
33/33 [==============================] - 0s 15ms/step - loss: 2.2848e-04 - val_loss: 0.0128
Epoch 9/100
33/33 [==============================] - 0s 15ms/step - loss: 2.0333e-04 - val_loss: 0.0120
Epoch 10/100
33/33 [==============================] - 0s 14ms/step - loss: 1

In [45]:
train_predict=model.predict(x_train_dl)
test_predict=model.predict(x_test_dl)
train_predict.shape, test_predict.shape

130/130 [==============================] - 0s 3ms/step


((16587, 5), (4147, 5))

In [46]:
test_pred_scale = (test_predict * (target_max_scale - target_min_scale)) + target_min_scale

In [47]:
y_test_scale = (y_test_dl * (target_max_scale - target_min_scale) + target_min_scale)

In [48]:
mean_squared_error(y_test_scale, test_pred_scale)

42937.401091054024

In [49]:
mean_absolute_error(y_test_scale, test_pred_scale)

162.89064281141256

In [50]:
r2_score(y_test_scale, test_pred_scale)

0.5306049278057218

In [51]:
import pickle

In [53]:
n_samples, n_fea = y_test_scale.shape
lstm_res = dict()
for idx, category in enumerate(ts_config.pred_features):
    cate_mse = mean_squared_error(y_test_scale[:, idx], test_pred_scale[:, idx])
    cate_mae = mean_absolute_error(y_test_scale[:, idx], test_pred_scale[:, idx])
    cate_rsqr = r2_score(y_test_scale[:, idx], test_pred_scale[:, idx])

    lstm_res[ts_config.pred_features[idx]] = [cate_mse, cate_mae, cate_rsqr]

    print(f'MSE in {ts_config.pred_features[idx]} = {cate_mse}')
    print(f'MAE in {ts_config.pred_features[idx]} = {cate_mae}')
    print(f'R2 score in {ts_config.pred_features[idx]} = {cate_rsqr}')
print(lstm_res)
with open(f'save/StackedLSTM/results.pkl', mode= 'wb') as writer:
    pickle.dump(lstm_res, writer)
writer.close()


MSE in high = 12680.739956872714
MAE in high = 94.24884140647791
R2 score in high = 0.8607032903348253
MSE in low = 40659.49873301695
MAE in low = 177.31289558500674
R2 score in low = 0.557618310880128
MSE in MA = 11776.560149289335
MAE in MA = 87.27457680103312
R2 score in MA = 0.8697294557127144
MSE in open = 39961.10743800331
MAE in open = 171.76461569529098
R2 score in open = 0.563397540957238
MSE in close = 109609.09917808768
MAE in close = 283.8522845692539
R2 score in close = -0.19842395885629438
{'high': [12680.739956872714, 94.24884140647791, 0.8607032903348253], 'low': [40659.49873301695, 177.31289558500674, 0.557618310880128], 'MA': [11776.560149289335, 87.27457680103312, 0.8697294557127144], 'open': [39961.10743800331, 171.76461569529098, 0.563397540957238], 'close': [109609.09917808768, 283.8522845692539, -0.19842395885629438]}


In [57]:
y_test_scale.shape

(4147, 5)

In [48]:
test_pred_scale.shape

(4147, 5)

(4147,)

In [52]:
n_samples, n_fea = y_test_scale.shape

In [54]:
y_test_scale[:, 1].reshape((n_samples, 1)).shape

(4147, 1)

## Stacked GRU

In [56]:
tf.keras.backend.clear_session()
gru_model=Sequential()
gru_model.add(GRU(16,return_sequences=True,input_shape=(ts_config.n_steps, ts_config.n_features)))
gru_model.add(GRU(16))
gru_model.add(Dense(ts_config.n_classes))
gru_model.compile(loss='mean_squared_error',optimizer='adam')

In [57]:
gru_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 7, 16)             1440      
                                                                 
 gru_1 (GRU)                 (None, 16)                1632      
                                                                 
 dense (Dense)               (None, 5)                 85        
                                                                 
Total params: 3,157
Trainable params: 3,157
Non-trainable params: 0
_________________________________________________________________


In [58]:
gru_model.fit(x_train_dl,y_train_dl,validation_data=(x_test_dl,y_test_dl),epochs=100,batch_size=512,verbose=1)
gru_model.save(f'save/GRU/gru_model.h5')

Epoch 1/100
33/33 [==============================] - 3s 27ms/step - loss: 0.0689 - val_loss: 0.0488
Epoch 2/100
33/33 [==============================] - 0s 13ms/step - loss: 0.0074 - val_loss: 0.0459
Epoch 3/100
33/33 [==============================] - 0s 14ms/step - loss: 0.0022 - val_loss: 0.0193
Epoch 4/100
33/33 [==============================] - 0s 13ms/step - loss: 5.1836e-04 - val_loss: 0.0103
Epoch 5/100
33/33 [==============================] - 0s 14ms/step - loss: 2.9041e-04 - val_loss: 0.0090
Epoch 6/100
33/33 [==============================] - 0s 13ms/step - loss: 2.4836e-04 - val_loss: 0.0087
Epoch 7/100
33/33 [==============================] - 0s 14ms/step - loss: 2.2596e-04 - val_loss: 0.0085
Epoch 8/100
33/33 [==============================] - 0s 14ms/step - loss: 2.0818e-04 - val_loss: 0.0086
Epoch 9/100
33/33 [==============================] - 0s 13ms/step - loss: 1.9199e-04 - val_loss: 0.0084
Epoch 10/100
33/33 [==============================] - 0s 14ms/step - loss: 1

In [77]:
train_predict_gru = gru_model.predict(x_train_dl)
test_predict_gru = gru_model.predict(x_test_dl)
print(train_predict_gru.shape, test_predict_gru.shape)

130/130 [==============================] - 0s 3ms/step
(16587, 5) (4147, 5)


In [78]:
gru_test_scale = (test_predict_gru * (target_max_scale - target_min_scale)) + target_min_scale

In [79]:
gru_test_scale

array([[3544.42915226, 3536.52338505, 3540.03738549, 3545.02631969,
        3540.72847373],
       [3539.28198213, 3530.62082072, 3538.97514263, 3539.28718372,
        3535.70343272],
       [3539.17958158, 3531.03887796, 3539.95091553, 3537.1200627 ,
        3534.50655977],
       ...,
       [4406.22285686, 4660.67464585, 4044.9562678 , 3829.32863538,
        3622.7433789 ],
       [4392.83725355, 4627.2413981 , 4034.45375058, 3823.19827873,
        3629.10811838],
       [4382.70255947, 4599.75801403, 4026.3625987 , 3819.4822985 ,
        3635.70498017]])

In [76]:

n_samples, n_fea = y_test_scale.shape
gru_res = dict()
for idx, category in enumerate(ts_config.pred_features):
    cate_mse = mean_squared_error(y_test_scale[:, idx], gru_test_scale[:, idx])
    cate_mae = mean_absolute_error(y_test_scale[:, idx], gru_test_scale[:, idx])
    cate_rsqr = r2_score(y_test_scale[:, idx], gru_test_scale[:, idx])

    gru_res[ts_config.pred_features[idx]] = [cate_mse, cate_mae, cate_rsqr]

    print(f'MSE in {ts_config.pred_features[idx]} = {cate_mse}')
    print(f'MAE in {ts_config.pred_features[idx]} = {cate_mae}')
    print(f'R2 score in {ts_config.pred_features[idx]} = {cate_rsqr}')
print(gru_res)
with open(f'save/GRU/results_gru.pkl', mode= 'wb') as writer:
    pickle.dump(gru_res, writer)
writer.close()


MSE in high = 12680.739956872714
MAE in high = 94.24884140647791
R2 score in high = 0.8607032903348253
MSE in low = 40659.49873301695
MAE in low = 177.31289558500674
R2 score in low = 0.557618310880128
MSE in MA = 11776.560149289335
MAE in MA = 87.27457680103312
R2 score in MA = 0.8697294557127144
MSE in open = 39961.10743800331
MAE in open = 171.76461569529098
R2 score in open = 0.563397540957238
MSE in close = 109609.09917808768
MAE in close = 283.8522845692539
R2 score in close = -0.19842395885629438
{'high': [12680.739956872714, 94.24884140647791, 0.8607032903348253], 'low': [40659.49873301695, 177.31289558500674, 0.557618310880128], 'MA': [11776.560149289335, 87.27457680103312, 0.8697294557127144], 'open': [39961.10743800331, 171.76461569529098, 0.563397540957238], 'close': [109609.09917808768, 283.8522845692539, -0.19842395885629438]}


In [77]:
gru_test_scale

array([[3536.27412001, 3516.42337976, 3538.13719616, 3514.26360426,
        3544.45772462],
       [3530.82863116, 3512.61676774, 3535.21561027, 3510.28237296,
        3539.67812644],
       [3528.35078445, 3509.94484734, 3535.72583082, 3508.10440525,
        3535.8166571 ],
       ...,
       [4027.68970103, 3769.36747687, 3915.4526918 , 3793.39303094,
        3838.60874058],
       [4028.92368942, 3769.44384561, 3923.57441869, 3795.94348413,
        3840.13813306],
       [4029.38955026, 3768.73829702, 3932.44308362, 3799.0249325 ,
        3841.66826557]])

In [78]:
y_test_scale

array([[3573.65007628, 3556.85003304, 3562.29623436, 3563.62002571,
        3557.53995056],
       [3589.81011779, 3564.09003568, 3563.2345051 , 3566.82004822,
        3564.67999336],
       [3575.89992754, 3556.54997781, 3564.23918094, 3564.68004395,
        3559.79999797],
       ...,
       [4105.14005328, 4086.87000917, 4001.32070697, 4103.03988975,
        4096.89007598],
       [4105.23998636, 4093.1898917 , 4002.93473328, 4096.87995119,
        4104.00002428],
       [4106.62991969, 4093.55005652, 4004.43386442, 4104.02003685,
        4093.72004005]])

## Our method

In [26]:
from model import TSModel
import tensorflow as tf
from tensorflow.keras.layers import Bidirectional, LSTM, Dense
from tensorflow.keras import Sequential

In [29]:
tf.keras.backend.clear_session()
bidir_model=Sequential()
bidir_model.add(Bidirectional(LSTM(16,return_sequences=True,input_shape=(ts_config.n_steps, ts_config.n_features))))
bidir_model.add(Bidirectional(LSTM(16)))
bidir_model.add(Dense(ts_config.n_classes))
bidir_model.compile(loss='mean_squared_error',optimizer='adam')

In [32]:
# bidir_model.summary()
bidir_model.fit(x_train_dl,y_train_dl,validation_data=(x_test_dl,y_test_dl),epochs=100,batch_size=512,verbose=1)

Epoch 1/100
33/33 [==============================] - 9s 56ms/step - loss: 0.0309 - val_loss: 0.0914
Epoch 2/100
33/33 [==============================] - 1s 18ms/step - loss: 0.0019 - val_loss: 0.0137
Epoch 3/100
33/33 [==============================] - 1s 20ms/step - loss: 2.4472e-04 - val_loss: 0.0143
Epoch 4/100
33/33 [==============================] - 1s 17ms/step - loss: 1.5637e-04 - val_loss: 0.0124
Epoch 5/100
33/33 [==============================] - 1s 17ms/step - loss: 1.3129e-04 - val_loss: 0.0115
Epoch 6/100
33/33 [==============================] - 1s 16ms/step - loss: 1.1312e-04 - val_loss: 0.0109
Epoch 7/100
33/33 [==============================] - 1s 17ms/step - loss: 9.9040e-05 - val_loss: 0.0103
Epoch 8/100
33/33 [==============================] - 1s 17ms/step - loss: 8.8116e-05 - val_loss: 0.0098
Epoch 9/100
33/33 [==============================] - 1s 17ms/step - loss: 7.9435e-05 - val_loss: 0.0092
Epoch 10/100
33/33 [==============================] - 1s 19ms/step - los

In [68]:
train_predict_bidir = bidir_model.predict(x_train_dl)
test_predict_bidir = bidir_model.predict(x_test_dl)
print(train_predict_bidir.shape, test_predict_bidir.shape)
bidir_model.save(f'save/BiLSTM_Time/model.h5')

130/130 [==============================] - 0s 4ms/step
(16587, 5) (4147, 5)


In [69]:
bidir_pred_scale = (test_predict_bidir * (target_max_scale - target_min_scale)) + target_min_scale

In [70]:
y_test_scale

array([[3573.65007628, 3556.85003304, 3562.29623436, 3563.62002571,
        3557.53995056],
       [3589.81011779, 3564.09003568, 3563.2345051 , 3566.82004822,
        3564.67999336],
       [3575.89992754, 3556.54997781, 3564.23918094, 3564.68004395,
        3559.79999797],
       ...,
       [4105.14005328, 4086.87000917, 4001.32070697, 4103.03988975,
        4096.89007598],
       [4105.23998636, 4093.1898917 , 4002.93473328, 4096.87995119,
        4104.00002428],
       [4106.62991969, 4093.55005652, 4004.43386442, 4104.02003685,
        4093.72004005]])

In [75]:

n_samples, n_fea = y_test_scale.shape
bidir_result = dict()
for idx, category in enumerate(ts_config.pred_features):
    cate_mse = mean_squared_error(y_test_scale[:, idx], bidir_pred_scale[:, idx])
    cate_mae = mean_absolute_error(y_test_scale[:, idx], bidir_pred_scale[:, idx])
    cate_rsqr = r2_score(y_test_scale[:, idx], bidir_pred_scale[:, idx])

    bidir_result[ts_config.pred_features[idx]] = [cate_mse, cate_mae, cate_rsqr]

    print(f'MSE in {ts_config.pred_features[idx]} = {cate_mse}')
    print(f'MAE in {ts_config.pred_features[idx]} = {cate_mae}')
    print(f'R2 score in {ts_config.pred_features[idx]} = {cate_rsqr}')
print(bidir_result)
with open(f'save/BiLSTM_Time/results_bidir.pkl', mode= 'wb') as writer:
    pickle.dump(bidir_result, writer)
writer.close()


MSE in high = 1786.8458572651514
MAE in high = 31.507823106917467
R2 score in high = 0.9803716699938331
MSE in low = 5312.98377266897
MAE in low = 55.781296917008355
R2 score in low = 0.9421939077249079
MSE in MA = 108465.93359541956
MAE in MA = 278.58631885526825
R2 score in MA = -0.19983390964606573
MSE in open = 13356.648659201868
MAE in open = 93.2332664953878
R2 score in open = 0.8540694684644325
MSE in close = 16775.75641565107
MAE in close = 108.98252168946306
R2 score in close = 0.8165802970081102
{'high': [1786.8458572651514, 31.507823106917467, 0.9803716699938331], 'low': [5312.98377266897, 55.781296917008355, 0.9421939077249079], 'MA': [108465.93359541956, 278.58631885526825, -0.19983390964606573], 'open': [13356.648659201868, 93.2332664953878, 0.8540694684644325], 'close': [16775.75641565107, 108.98252168946306, 0.8165802970081102]}
